# Ego4D Moments Benchmark (NLQ) Quickstart

Please set your resources to GPU (Runtime -> Change runtime type -> GPU).

This quickstart will show:
1. An overview of the training data
2. How to train the baseline (VSLNet)

To begin: add your **access keys** below, change your Runtime Type to **GPU**, and run cells **one by one** as you read through. This helps avoid timeouts since Colab gives more GPU cycles to interactive notebooks.

## Resources
- [Baseline Repo](https://github.com/EGO4D/episodic-memory/tree/main/NLQ/VSLNet)
- [Docs](https://ego4d-data.org/docs/benchmarks/episodic-memory/)
- [EvalAI Challenge](https://eval.ai/web/challenges/challenge-page/1629/overview)

## Setup Google Colab


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!dir drive/MyDrive/data/

ego4d_data_v1  ego4d_data_v2


### Check Downloaded Files

In [ ]:
# Ensure we have downloaded the files correctly
!ls /content/drive/MyDrive/data/ego4d_data_v1/v1/annotations | grep nlq

nlq_test_unannotated.json
nlq_train.json
nlq_val.json


In [ ]:
!ls /content/drive/MyDrive/data/ego4d_data_v1/v1/omnivore_video_swinl_fp16 | wc -l

1260


### Clone the Episodic Memory Baseline Repository

In [ ]:
%%bash
git clone https://github.com/EGO4D/episodic-memory
cd episodic-memory
git pull
git checkout nlq_fixes_and_fp16_support

Already up to date.
Branch 'nlq_fixes_and_fp16_support' set up to track remote branch 'nlq_fixes_and_fp16_support' from 'origin'.


Cloning into 'episodic-memory'...
Switched to a new branch 'nlq_fixes_and_fp16_support'


# Prepare Dataset

The NLQ baseline repository for VSLNet requires you to prepare the data for training and evaluation purposes. From the [README.md](https://github.com/EGO4D/episodic-memory/blob/main/NLQ/VSLNet/README.md#preparation) we need to run the `prepare_ego4d_dataset.py` script.

### Setup Environment Variables for NLQ

First let's setup some environment variables and setup the paths as NLQ's scripts will expect.

In [ ]:
with open("vars.sh", "w") as out_f:
  out_f.write("""
export NAME=omnivore_video_fp16
export TASK_NAME=nlq_official_v1_$NAME
export BASE_DIR=data/dataset/nlq_official_v1_$NAME
export FEATURE_BASE_DIR=data/features/nlq_official_v1_$NAME/
export FEATURE_DIR=$FEATURE_BASE_DIR/video_features
export MODEL_BASE_DIR=/content/nlq_official_v1/checkpoints/

cd episodic-memory/NLQ/VSLNet
"""
  )

In [ ]:
%%bash

source vars.sh

echo $FEATURE_BASE_DIR
mkdir -p $FEATURE_BASE_DIR
ln -s /content/drive/MyDrive/data/ego4d_data_v1/v1/omnivore_video_swinl_fp16 $FEATURE_DIR

data/features/nlq_official_v1_omnivore_video_fp16/


In [ ]:
%%bash
%%capture

source vars.sh
pip install nltk submitit torch torchaudio torchvision tqdm transformers tensorboard Pillow terminaltables

# Train and Evaluation

## Run the Prepare Script

This script will take a while to run and may not output progress until it is done. Please be patient.

In [ ]:
%%bash

source vars.sh

python utils/prepare_ego4d_dataset.py \
    --input_train_split /content/drive/MyDrive/data/ego4d_data_v1/v1/annotations/nlq_train.json \
    --input_val_split /content/drive/MyDrive/data/ego4d_data_v1/v1/annotations/nlq_val.json \
    --input_test_split /content/drive/MyDrive/data/ego4d_data_v1/v1/annotations/nlq_test_unannotated.json \
    --video_feature_read_path $FEATURE_DIR \
    --clip_feature_save_path $FEATURE_BASE_DIR/official \
    --output_save_path $BASE_DIR

## Train a Model

Please note:
1. These are *not* the parameters for the original baseline model in the Ego4D whitepaper.
2. Omnivore video features are used (slowfast was originally used), and their FP16 variant. This is due to free colab constraints (100GB disk space).

Omnivore video features do out-perform slowfast features.

In [ ]:
%load_ext tensorboard

You may have to re-run this cell after you run the training script. You can try to reload data.

In [ ]:
!mkdir -p /content/episodic-memory/NLQ/VSLNet/runs/
%tensorboard --logdir /content/episodic-memory/NLQ/VSLNet/runs/

Unfortunately due to colab and the time taken in the below script (it first saves additional metadata to disk) - the cell below takes a while to get started. Please be patient when running it may take at least 30 minutes. You may get timed out from colab.

You can tell if the training is started by inspecting the filesystem on the left hand side. The directory: `episodic-memory/NLQ/VSLNet/runs` will populate with a subdirectory for the tensorboard logdir.

Please note, these are *not* the hyper parameters used for the baseline. The following uses an aggressive learning rate and a low number of epochs, to converge faster so you don't have to wait as long for this cell to finish. :)

In [ ]:
!ls /content/drive/MyDrive/data/ego4d_data_v1/v1/annotations/nlq_val.json

/content/drive/MyDrive/data/ego4d_data_v1/v1/annotations/nlq_val.json


### Test Mode

In [ ]:
%%bash

source vars.sh

# machine parameters
export DATALOADER_WORKERS=1
export NUM_WORKERS=2
export VAL_JSON_PATH="/content/drive/MyDrive/data/ego4d_data_v1/v1/annotations/nlq_val.json"

# hyper parameters
export BATCH_SIZE=32
export DIM=128
export NUM_EPOCH=10
export MAX_POS_LEN=128
export INIT_LR=0.0025

export TB_LOG_NAME="${NAME}_bs${BATCH_SIZE}_dim${DIM}_epoch${NUM_EPOCH}_ilr${INIT_LR}"

python main.py \
    --task $TASK_NAME \
    --predictor bert \
    --dim $DIM \
    --mode train \
    --video_feature_dim 1536 \
    --max_pos_len $MAX_POS_LEN \
    --init_lr $INIT_LR \
    --epochs $NUM_EPOCH \
    --batch_size $BATCH_SIZE \
    --fv official \
    --num_workers $NUM_WORKERS \
    --data_loader_workers $DATALOADER_WORKERS \
    --model_dir $MODEL_BASE_DIR/$NAME \
    --eval_gt_json $VAL_JSON_PATH \
    --log_to_tensorboard $TB_LOG_NAME \
    --tb_log_freq 5 \
    --remove_empty_queries_from train

Running with Namespace(save_dir='datasets', task='nlq_official_v1_omnivore_video_fp16', eval_gt_json='/content/drive/MyDrive/data/ego4d_data_v1/v1/annotations/nlq_val.json', fv='official', max_pos_len=128, num_workers=2, data_loader_workers=1, word_size=None, char_size=None, word_dim=300, video_feature_dim=1536, char_dim=50, dim=128, highlight_lambda=5.0, num_heads=8, drop_rate=0.2, predictor='bert', gpu_idx='0', seed=12345, mode='train', epochs=10, batch_size=32, num_train_steps=None, init_lr=0.0025, clip_norm=1.0, warmup_proportion=0.0, extend=0.1, period=100, text_agnostic=False, video_agnostic=False, model_dir='/content/nlq_official_v1/checkpoints//omnivore_video_fp16', model_name='vslnet', suffix=None, log_to_tensorboard='omnivore_video_fp16_bs32_dim128_epoch10_ilr0.0025', tb_log_dir='./runs', tb_log_freq=5, slurm=False, slurm_wait=False, slurm_partition='pixar', slurm_constraint='volta', slurm_gpus=1, slurm_cpus=10, slurm_timeout_min=720, slurm_log_folder='slurm_log', remove_empt

2025-04-30 14:26:13.939210: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746023173.960827   14574 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746023173.967956   14574 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-30 14:26:13.989665: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
process episodic nlq test: 100%|██████████| 333/333 [00:00<00:00, 57920.84it/s]
/usr/local/lib/python3.11/dist-packages/huggi

### Test Mode

In [ ]:
  %%bash

  source vars.sh

  # machine parameters
  export DATALOADER_WORKERS=1
  export NUM_WORKERS=2
  export VAL_JSON_PATH="/content/drive/MyDrive/data/ego4d_data_v1/v1/annotations/nlq_val.json"

  # hyper parameters
  export BATCH_SIZE=32
  export DIM=128
  export NUM_EPOCH=10
  export MAX_POS_LEN=128
  export INIT_LR=0.0025

  export TB_LOG_NAME="${NAME}_bs${BATCH_SIZE}_dim${DIM}_epoch${NUM_EPOCH}_ilr${INIT_LR}"

  python main.py \
      --task $TASK_NAME \
      --predictor bert \
      --dim $DIM \
      --mode test \
      --video_feature_dim 1536 \
      --max_pos_len $MAX_POS_LEN \
      --init_lr $INIT_LR \
      --epochs $NUM_EPOCH \
      --batch_size $BATCH_SIZE \
      --fv official \
      --num_workers $NUM_WORKERS \
      --data_loader_workers $DATALOADER_WORKERS \
      --model_dir $MODEL_BASE_DIR/$NAME \
      --eval_gt_json $VAL_JSON_PATH \
      --log_to_tensorboard $TB_LOG_NAME \
      --tb_log_freq 5 \
      --remove_empty_queries_from test

Running with Namespace(save_dir='datasets', task='nlq_official_v1_omnivore_video_fp16', eval_gt_json='/content/drive/MyDrive/data/ego4d_data_v1/v1/annotations/nlq_test_unannotated.json', fv='official', max_pos_len=128, num_workers=2, data_loader_workers=1, word_size=None, char_size=None, word_dim=300, video_feature_dim=1536, char_dim=50, dim=128, highlight_lambda=5.0, num_heads=8, drop_rate=0.2, predictor='bert', gpu_idx='0', seed=12345, mode='test', epochs=10, batch_size=32, num_train_steps=None, init_lr=0.0025, clip_norm=1.0, warmup_proportion=0.0, extend=0.1, period=100, text_agnostic=False, video_agnostic=False, model_dir='/content/nlq_official_v1/checkpoints/omnivore_video_fp16', model_name='vslnet', suffix=None, log_to_tensorboard='omnivore_video_fp16_bs32_dim128_epoch10_ilr0.0025', tb_log_dir='./runs', tb_log_freq=5, slurm=False, slurm_wait=False, slurm_partition='pixar', slurm_constraint='volta', slurm_gpus=1, slurm_cpus=10, slurm_timeout_min=720, slurm_log_folder='slurm_log', 

2025-04-30 15:51:09.337604: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746028269.359545   36782 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746028269.366469   36782 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-30 15:51:09.390388: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
load video features: 100%|██████████| 1659/1659 [01:38<00:00, 16.88it/s]
/usr/local/lib/python3.11/dist-packages/huggingface_